In [1]:
print("CardiAP v1.0")

CardiAP v1.0


In [2]:
from IPython.display import HTML, display, clear_output

import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, Text, HBox, VBox, Image

import base64
import cv2

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from lib.image import image_data
from lib.analysis import dyssynchrony_analysis

import warnings

pd.set_option('display.max_rows', 20)
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [3]:
def load_cv2_image_from_bytes(bytes_):
    return cv2.imdecode(np.frombuffer(bytes_, dtype=np.uint8), cv2.IMREAD_COLOR)

def get_uploader_contents(uploader):
    return [uploader.value[file]["content"] for file in list(uploader.value)]

def display_html(html):
    display(widgets.HTML(value=html))
    
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
    
def create_inttext(description, min = None, max = None):
    return widgets.BoundedIntText(
        value=0,
        min = min,
        max = max,
        description=description,
        disabled=False
    )
    
def plot_analysis_result_slice(df,slice_number):
    
    fig, ((ax1, ax2)) = plt.subplots(2, figsize=(20,10))

    ax1.plot(df.loc[slice_number, "amplitudes"])
    ax1.set_title('amplitudes')
    ax1.grid()

    # log y axi
    ax2.plot(df.loc[slice_number, "intensities"])
    ax2.set_title('Intensities')
    ax2.grid()
    
    plt.show()    

def read_progress_img():
    animatedGif = "./assets/progress.gif"
    with open(animatedGif , "rb") as file:
        return file.read()

def create_image(image):
    return Image(
        value=image,
        format='gif',
        width=100,
        height=100)

 
def render_image_visualization(analysis_results):
    render_analysis_results_visualization(analysis_results, 
                                          title = "Peaks",
                                          kind = "image",
                                          download_filename = "complete_cell",
                                          progress = without_progress)


def render_slice_visualization(analysis_results, slice_number):
    render_analysis_results_visualization(analysis_results, 
                                          title = "Slice",
                                          kind = "slices",
                                          download_filename = f"slice_{slice_number}",
                                          progress = with_progress,
                                          slice_number = slice_number)    
    
def render_analysis_results_visualization(analysis_results, title, kind, download_filename, progress, slice_number = 0):
    """
    Renders analysis results into three sections: 
    - Peaks table
    - Table download
    - Peaks plot
    """
    display_html(f"<h3>{title} table</h3>")
    table = build_analysis_results_table(analysis_results, kind = kind)
    table = table[table.slice_number == slice_number]
    table = table.drop(columns = "slice_number")
    display(table)
    
    progress(lambda: display(create_download_link(table, title = f"Download {title} Table", filename = f"{download_filename}.csv")))    

    display_html(f"<h3>{title} plot</h3>")
    progress(lambda: plot_analysis_result_slice(analysis_results[kind], slice_number))    
    
    
def display_results(analysis_results):
    """
    Displays the results into three separate tabs. 
    Every print or display will be automatically redirected to it. 
    """
    
    complete_cell_output = widgets.Output()
    slices_general_output = widgets.Output()
    slices_detail_output = widgets.Output()
    
    tab = widgets.Tab()
    tab.children = [complete_cell_output, slices_general_output, slices_detail_output]
    tab.set_title(0, "Complete Cell")
    tab.set_title(1, "Slices General Data")
    tab.set_title(2, "Slices Detail")
    display(tab)

    with complete_cell_output:
        display_html("<h2>Complete Cell</h2>")
        render_image_visualization(analysis_results)

    with slices_general_output:    
        display_html("<h2>Slices General Data</h2>")
        table = build_analysis_results_table(analysis_results, kind = 'slices')
        display(
            create_download_link(
                build_analysis_results_table(analysis_results, kind = 'slices'), 
                title = "Download Slices Table", 
                filename = "slices.csv"))    

    with slices_detail_output:        
        display_html("<h2 style='margin-top:20px; margin-botton:15px'>Slices Detail</h2>")
        slice_selector = create_inttext("Plot slice: ", min = 0, max = len(analysis_results["slices"]) - 1)
        plot_btn = widgets.Button(description="Show details", button_style='danger')
        plot_box = widgets.Output()

        @plot_box.capture()
        def on_button_clicked(_button):
            plot_box.clear_output()
            render_slice_visualization(analysis_results, slice_selector.value)

        plot_btn.on_click(on_button_clicked)    
        display(slice_selector)
        display(plot_btn)
        display(plot_box)

def without_progress(code):
    code()        
        
def with_progress(code):
    progress = create_image(read_progress_img())
    display(progress)
    code()
    progress.close()
    
def clear_uploader(uploader):
    uploader.value.clear()
    uploader._counter = 0

def cardiaAp_analyzer(image):
    return dyssynchrony_analysis.analyze_image(image, \
                                               min_dist_between_maxs.value, \
                                               calibration.value, \
                                               slice_width=slice_width.value)

def run_cardiap_pipeline(content):
    """
    Loads a cell image from uploaded content bytes
    and runs it against the cardiap analyzer. 
    
    This function returns the raw cardiap results dictionary and two pandas dataframes: 
    
    - image: the whole cell image analysis results  
    - slices: the sliced analysis results 
    
    """
    raw_results = cardiaAp_analyzer(load_cv2_image_from_bytes(content))
    return {
        "raw": raw_results,
        "image": pd.DataFrame([raw_results["image"]]),
        "slices":  pd.DataFrame(raw_results["slices"])
    }

def transpose_results_table(df):
    result = df[["amplitudes", "max_peaks_positions", "max_peaks_intensities", "times_to_peaks",  "times_to_half_peaks", "tau_s"]].apply(pd.Series.explode).reset_index()
    result = result.rename(columns = {"index": "slice_number"})
    result = result.astype({
        "amplitudes":"float", 
        "max_peaks_positions":"int",
        "max_peaks_intensities":"int",
        "times_to_peaks": "int", 
        "times_to_half_peaks": "int",
        "tau_s": "float"
     })
    return result

def build_analysis_results_table(analysis_results, kind):
    """
    Build the final output analysis results table.
    This function works both for building slices table - where you will get a row for each slice and position - 
    or and for building image table - where you get a single row for each position
    Final tables contains the same columns as raw analysis results, except by min_* columns and amplitudes
    """
    return transpose_results_table(analysis_results[kind])
    

In [4]:
uploader = widgets.FileUpload(accept='.tif', multiple=True)

slice_width = widgets.IntText(
    value=1,
    description='Slice width:',
    disabled=False
)

min_dist_between_maxs = widgets.IntText(
    value=0,
    description='Dist between maxs:',
    disabled=False
)

calibration = widgets.IntText(
    value=0,
    description='Calibration:',
    disabled=False
)

html_text_upload = "<p style='font-size:15px; color:black'> Upload your image to initialize the analysis</p>"
header_text_upload = widgets.HTML(value=html_text_upload)

box_upload = Layout(display='flex',
                    flex_flow='flex-wrap',
                    width='50%',
                    margin='0px 10px 10px 20px',
                    padding=' 2px 5px 0 5px',
                    justify_content='space-between')

vbox_upload = widgets.VBox([header_text_upload, uploader], layout=box_upload)

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='50%',
                    margin='0px 10px 10px 20px',
                    padding=' 2px 5px 0 5px',
                    justify_content='space-between')


analyze_btn = widgets.Button(description="Analyze", button_style='danger', layout=Layout(margin='10px 0px 0px 0px'))
results_box = widgets.Output()

@results_box.capture()
def on_button_clicked(_button):
    """
    Runs analysis and displays results. 
    It stores them in __last_analysis_results__
    for debugging purposes
    """
    global __last_analysis_results__
    results_box.clear_output()

    if uploader._counter == 0:
        print("Please upload an image first")
    else:
        children = []
        cells_count = uploader._counter

        for index, content in enumerate(get_uploader_contents(uploader), start = 1):
            print(f"Analyzing cell {index} of {cells_count}. Please wait...")
        
            output = widgets.Output()
            children.append(output)
            
            with output:
                __last_analysis_results__ = run_cardiap_pipeline(content)
                display_results(__last_analysis_results__)
            
            uploader._counter -= 1

            
        clear_uploader(uploader)
        results_box.clear_output()

        results_tab = widgets.Tab()
        results_tab.children = children
        display(results_tab)
        for index, _ in enumerate(children):
            results_tab.set_title(index, f"Cell {str(index + 1)}")

        
analyze_btn.on_click(on_button_clicked)

vbox_settings = widgets.VBox([slice_width, min_dist_between_maxs, calibration, analyze_btn], layout=box_layout)


In [5]:
h1_main_section = widgets.HTML("""
    <h1 style='font-size:25px; color:black'>
        Welcome to Cardi<b>AP</b>
    </h1>""")
paragraph_description_section = widgets.HTML("""
    <p style='font-size:15px; color:black'> 
        <b>CardIAP</b> is an open-source web application for analyzing series of
        calcium handling phenomena from confocal microscopy images. CardIAP is a Python based tool,
        which allows users to easily work with a single file or a pool of images,
        and obtain representative amplitude and kinetics data. 
        <br>Copyright (c) 2020-2021 Velez Rueda, Garcia Smith, Sommese</br>
    </p>""")

athors_section_title = widgets.HTML("""
    <h3 style='font-size:25px; color:black'>
        Authors
    </h3>""")

athors_section_description = widgets.HTML("""
    <p style='font-size:15px; color:black'> 
        So far, <b>CardIAP</b> was developed by <a href=¨https://github.com/AJVelezRueda¨>Ana Julia Velez Rueda</a> (UNQ-CONICET),
        <a href=¨https://github.com/agusgs¨>Agustín Garcia Smith</a> (UNQ) &
        <a href=¨https://github.com/orgs/leandrosommese¨>Leandro M. Sommese</a> (UNQ-CONICET). 
        
        <br>If you want to be part of this project and contribute please contact us.</br>
    </p>""")


citations_section_title = widgets.HTML("""
    <h3 style='font-size:25px; color:black'>
        Citations
    </h3>""")

citations_section_description = widgets.HTML("""
    <p style='font-size:15px; color:black'> 
        When using  <b>CardIAP</b> , please cite:
        Ana Julia Velez Rueda, Agustín García Smith, Luis Alberto Gonano, Gustavo Parisi and Leandro Matías Sommese. CardIAP: Calcium images analyzer.
    </p>""")

citations_section_title = widgets.HTML("""
    <h3 style='font-size:25px; color:black'>
        Usage
    </h3>""")

citations_section_description = widgets.HTML("""
    <p style='font-size:15px; color:black'> 
        Please see documentation and usage information in our <a href="https://cardiap.github.io/"><b>home page</b></a>.
    </p>""")

Runing_section = widgets.HTML("""
    <h3 style='font-size:25px; color:black'> 
        Run your job 
    </h3>
    """)



layouts_intro = Layout(display='flex',
                    flex_flow='flex-wrap',
                    flex_direction='column',
                    width='50%',
                    margin='0px 10px 5px 20px',
                    padding=' 2px 5px 0 5px',
                    justify_content='space-between')

vbox_main_section = widgets.VBox([h1_main_section, 
                                  paragraph_description_section, 
                                  athors_section_title, 
                                  athors_section_description, 
                                  citations_section_title, 
                                  citations_section_description, Runing_section], layout=layouts_intro)



In [6]:
display(vbox_main_section)

#display_html("<h2 style='font-size:25px; color:black'> Run your job </h2>")


display(vbox_upload)
display(vbox_settings)

display(results_box)

Output()